In [1]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import torch
import torch.nn.functional as F
import numpy as np

from ssgd import StreamingSGD

# Model definition

A StreamingSGD compatible model now needs to be able to "detach" layers as well as gather input/output and gradients. It also needs a list of layers. See below for implementation example. In the future we want to implement this using PyTorch hooks and modules.

In [ ]:
class ExampleNet(torch.nn.Module):
    def __init__(self):
        super(ExampleNet, self).__init__()
        
        self.input_layer = torch.nn.Conv2d(3, 3, kernel_size=3)
        self.layers = [self.input_layer]

        for i in range(5):  # use 9 for 8194 x 8194 images
            self.add_block(i)
        
        final_conv5 = torch.nn.Conv2d(3, 1, kernel_size=10)
        self.add_module("final", final_conv5)
        
        self.layers.extend([final_conv5])
        
    def add_block(self, i):
        conv1 = torch.nn.Conv2d(3, 3, kernel_size=3)
        conv2 = torch.nn.Conv2d(3, 3, kernel_size=3)
        conv3 = torch.nn.Conv2d(3, 3, kernel_size=3)
        maxpool = torch.nn.MaxPool2d(2, stride=2)
        
        self.add_module("conv1-" + str(i), conv1)
        self.add_module("conv2-" + str(i), conv2)
        self.add_module("conv3-" + str(i), conv3)
        self.add_module("maxpool-" + str(i), maxpool)

        self.layers.extend([conv1, conv2, conv3, maxpool])
        
    def backward(self, gradient):
        self.gradients = []
        for i, output in reversed(list(enumerate(self.output))):
            if i < (len(self.output) - 1):
                gradient = self.input[i+1].grad
            output.backward(gradient=gradient, retain_graph=True)
            self.gradients.append(gradient)

    def forward(self, x, stop_index=-1, start_index=0, detach=False):
        if detach:
            self.output = []
            self.input = []
        for i, layer in enumerate(self.layers[start_index:]):
            if detach:
                x = torch.autograd.Variable(x.data, requires_grad=detach)
                self.input.append(x)
            if i == stop_index:
                break
            if i + 1 == len(self.layers[start_index:]):
                x = layer(x)
                x = x.view(-1, 1)
                x = F.sigmoid(x)
            else:
                x = F.relu(layer(x))
            if detach:
                self.output.append(x)
        return x

model = ExampleNet()
# model = model.double()

Weight initialization; we use positive values to generate large gradients, better for testing if final gradients are correct.

In [ ]:
for i, layer in enumerate(model.modules()):
    if isinstance(layer, torch.nn.Conv2d):
        layer.weight.data.fill_(0.04)
        layer.bias.data.zero_()

In [ ]:
print(model)

In [ ]:
sCNN = StreamingSGD(model)

# Configurations

In [ ]:
stop_index = 13  # use 21 for 8194x8194 images
img_size = 514  # try 8194, see last segment for details

cuda = False  # execute this notebook on the GPU
verbose = True  # enable / disable logging
divide_in = 2  # tip: use 25 for 8194x8194 when memory constraint

# Configure streaming SGD

In [ ]:
if cuda:
    model.cuda()

In [ ]:
sCNN.configure(model.layers, stop_index, (img_size, img_size, 3), divide_in, cuda, verbose)

# Generate random image and fake label

In the current implementation the whole image needs to be able to fit in memory (RAM).

In [ ]:
image = torch.FloatTensor(3, img_size, img_size).normal_(0, 1)
target = torch.FloatTensor(1, 1).fill_(0)

# image = image.double()
# target = target.double()

if cuda:
    target = target.cuda()

In [ ]:
image_var = torch.autograd.Variable(image)
# image_var = image_var.double()

In [ ]:
criterion = torch.nn.BCELoss()

In [ ]:
output, feature_map = sCNN.forward(image_var)

In [ ]:
output

In [ ]:
loss = criterion(output, torch.autograd.Variable(target)); loss

In [ ]:
full_gradients = sCNN.backward(image_var, feature_map, loss)

"Everything filled" means that all gradients were reconstructed succesfully!

Save the gradients of the conv2d layer to compare with normal SGD:

In [ ]:
streaming_conv_gradients = []

for i, layer in enumerate(model.layers):
    if isinstance(layer, torch.nn.Conv2d):
        if layer.weight.grad is not None:
            streaming_conv_gradients.append(layer.weight.grad.clone()) 

# Compare to normal SGD

Reset the gradients and perform a normal for backward pass.

In [ ]:
for i, layer in enumerate(model.layers):
    if isinstance(layer, torch.nn.Conv2d):
        if layer.weight.grad is not None:
            layer.weight.grad.data.zero_()
            layer.bias.grad.data.zero_()

In [ ]:
output = model.forward(image_var[None]);

In [ ]:
output

This output should be the same as the streaming SGD output, if so the loss will also be the same:

In [ ]:
loss = criterion(output, torch.autograd.Variable(target)); loss

Here we do a normal PyTorch backward pass:

In [ ]:
loss.backward()

# Compare the gradients of the conv2d layers

In [ ]:
normal_conv_gradients = []

for i, layer in enumerate(model.layers):
    if isinstance(layer, torch.nn.Conv2d):
        if layer.weight.grad is not None:
            normal_conv_gradients.append(layer.weight.grad) 

In [ ]:
for i in range(len(streaming_conv_gradients)):
    max_diff = torch.max(torch.abs(streaming_conv_gradients[i].data - 
                      normal_conv_gradients[i].data))
    print("Layer", i, "\taverage gradient size:", torch.mean(streaming_conv_gradients[i].data))

In [ ]:
for i in range(len(streaming_conv_gradients)):
    max_diff = torch.max(torch.abs(streaming_conv_gradients[i].data - 
                      normal_conv_gradients[i].data))
    print("Layer", i, "\tmax difference between gradients:", max_diff)

As you can see the difference of the gradients of the conv2d layers between the methods is (almost) numerically equivalent. The small differences are because of loss of significance with the floating points calculations. 

---

# Things to try:

* Use doubles instead of floats to reduce the difference (use model.double() and image_var.double())
* Make the image bigger than would fit on a GPU 
    - e.g. 8194x8194, make sure to add 3 more blocks in the model (see comments)
* If you want you can compare the reconstructed input gradients of each layer: 
    - pass fill_gradient=True in backward() function
    - compare full_gradients with self.model.gradients after the full model backward pass.
* For testing purposes the number of filters is small in this notebook, try increasing them